In [24]:
import pandas as pd
import numpy as np
import json
import glob
import re

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from wordcloud import WordCloud, STOPWORDS

#spacy
import spacy
#from nltk.corpus import STOPWORDS

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings


In [25]:
DATA_PATH = 'data/wordclouds/unfiltered_polarity.csv'
df = pd.read_csv(DATA_PATH)
df.sample(5)

,Unnamed: 0,Artist,Song Name,release date,GPE,Line,Line Before,Line After,polarity,pos,neg,neu,compound
3407,3407,Nas,Back When,NaN,New York,New York-New York-New York-New York,"""Warning-Warning-Warning-Warning",The sound-The sound-The sound,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.000,1.000,0.0000
4268,4268,Ol' Dirty Bastard,Go Go Go,NaN,New York,"Yo East New York in this motherfucker, Blahzay...","I said, have you heard a—Embed', 'Go Go Go Lyr...","Brooklyn in this motherfucker, Dirt Dog in thi...","{'neg': 0.209, 'neu': 0.791, 'pos': 0.0, 'comp...",0.000,0.209,0.791,-0.4404
3128,3128,Mos Def,Mathematics (dirty),NaN,America,Nearly half of America\'s largest cities is on...,Spent on national defense but folks still live...,That\'s why they gave Ricky Ross all the crack,"{'neg': 0.256, 'neu': 0.476, 'pos': 0.268, 'co...",0.268,0.256,0.476,-0.2023
1958,1958,Cam'ron,Bum Bum,2002-05-14,Jamaica,I come fi nice up Jamaicans,MC is my ambition,So Bum Bum,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.000,1.000,0.0000
2440,2440,Wu-Tang Clan,Dog Shit,1993-11-09,New York,"A New York Islander, my clique\'s NY Giants",And these habitual line steppers can die trying,A pocket full of NY (K)nicks for my clients,"{'neg': 0.358, 'neu': 0.642, 'pos': 0.0, 'comp...",0.000,0.358,0.642,-0.5994


In [26]:
df['release date'] = pd.to_datetime(df['release date'])
df

,Unnamed: 0,Artist,Song Name,release date,GPE,Line,Line Before,Line After,polarity,pos,neg,neu,compound
0,0,French Montana,100,2012-04-13,Harlem,Shout out to ma niggas in Harlem,"Motthaven, you know, Tremont",Can't call no blocks out there,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.0,1.000,0.0000
1,1,French Montana,2 Times,NaT,New York,New York undercovers amongst the others,Still watching out for JC and Torres,The family be couple hundred of us,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.0,1.000,0.0000
2,2,French Montana,50's & 100's,2016-11-05,Bronx,Who run it? (South Bronx),"40 cars, 40 chains2Embed"", ""50's & 100's Lyric...","Who run it? (North Memphis, let me chirp these...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.0,1.000,0.0000
3,3,French Montana,50's & 100's,2016-11-05,South Bronx,Who run it? (South Bronx),"40 cars, 40 chains2Embed"", ""50's & 100's Lyric...","Who run it? (North Memphis, let me chirp these...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.0,1.000,0.0000
4,4,French Montana,Bag,NaT,New York,"Talk about New York, big estate","And grab a casket, my chest christens in the l...",Got the bank and I ain't fit (That's a stash),"{'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'comp...",0.286,0.0,0.714,0.4939
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4336,4336,Ol' Dirty Bastard,Raw Hide,2005-01-04,America,"All the people in America, all the people outs...","See Wu-Tang'd like to thank, all the people ac...",For listenin' to our music,"{'neg': 0.0, 'neu': 0.643, 'pos': 0.357, 'comp...",0.357,0.0,0.643,0.6124
4337,4337,Ol' Dirty Bastard,Raw Hide (2020 Remaster),2002-03-19,America,"All the people in America, all the people outs...","See Wu-Tang'd like to thank, all the people ac...",For listenin' to our music,"{'neg': 0.0, 'neu': 0.643, 'pos': 0.357, 'comp...",0.357,0.0,0.643,0.6124
4338,4338,Ol' Dirty Bastard,Recognize,1995-03-28,Broadway,"(Murder Avenue L trains, Broadway blackouts",It go back like coal ovens and ice boxes,"Brooklyn Zu keep history, fam shake the trends","{'neg': 0.0, 'neu': 0.762, 'pos': 0.238, 'comp...",0.238,0.0,0.762,0.3612
4339,4339,Ol' Dirty Bastard,Recognize,1995-03-28,Brooklyn,"Brooklyn Zu keep history, fam shake the trends","(Murder Avenue L trains, Broadway blackouts",Five-hundred years of hard-working bodies,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.0,1.000,0.0000


In [27]:
df['year'] = pd.DatetimeIndex(df['release date']).year
df['decade'] = df.year - df.year.values%10
df.drop(columns='polarity', inplace=True)
df.sort_values(by='decade', inplace=True)

In [28]:
negative = df.sort_values(by=['decade','compound', 'GPE'], ascending=True)
positive = df.sort_values(by=['decade', 'compound', 'GPE'], ascending=False)

In [29]:
negative.head(10)

,Unnamed: 0,Artist,Song Name,release date,GPE,Line,Line Before,Line After,pos,neg,neu,compound,year,decade
2355,2355,Big Daddy Kane,Bed Stuy Represent,1989-09-19,New York,I made the streets of New York hot as a SAUNA,So bitches wouldn\'t have to go out and rob,And hung a Home Sweet Ho sign on every corner,0.00,0.527,0.473,-0.8313,1989.0,1980.0
457,457,Kurtis Blow,The Bronx,1981-01-01,Bronx,It's called the Bronx,I'll be a Bronx boy 'til the day I die,The Bronx,0.00,0.358,0.642,-0.5994,1981.0,1980.0
1561,1561,Naughty by Nature,World Go Round,1989-12-05,Brooklyn,"A Brooklyn boy dies, shot by a cop for a play gun","From the cradle to the killa, leavin' corpse a...","Our kids' days are up, even if they ain't stra...",0.00,0.291,0.709,-0.5719,1989.0,1980.0
436,436,Kurtis Blow,Nervous,1984-01-01,Harlem,"Nervous, nervous (Wo!)Embed"", ""One-Two-Five (M...","Nervous, nervous (Nuh-nervous)",Where the scene is tough and the folks are hip,0.00,0.808,0.192,-0.4939,1984.0,1980.0
977,977,Public Enemy,Get It In,1988-06-28,Bronx,"In the Bronx, we rock the block, you gotta (Ge...","Riding on the block, you gotta (Get it in)","Nassau county on the rock, you gotta (Get it in)",0.00,0.266,0.734,-0.4404,1988.0,1980.0
429,429,Kurtis Blow,America,1980-09-29,America,Take American hostages across the seas,Revolutionary terrorists swarm like bees,But how long Reagan go for that?,0.26,0.427,0.312,-0.3818,1980.0,1980.0
1367,1367,Run–D.M.C.,Queens Day,1984-03-27,Queens,"Fly from Shirt Kings, from Queensbridge to 118","Niggas from Rochdale popping shells, snatching...","I had my eyes open, stepping out into the world",0.00,0.286,0.714,-0.3400,1984.0,1980.0
1366,1366,Run–D.M.C.,Queens Day,1984-03-27,Queensbridge,"Fly from Shirt Kings, from Queensbridge to 118","Niggas from Rochdale popping shells, snatching...","I had my eyes open, stepping out into the world",0.00,0.286,0.714,-0.3400,1984.0,1980.0
1372,1372,Run–D.M.C.,Queens Day,1984-03-27,Queens,"Queensbridge and Astoria, too, the story is true",The young guns and Lost Boys made the news,So let's party for the borough and shit,0.00,0.223,0.777,-0.3182,1984.0,1980.0
1371,1371,Run–D.M.C.,Queens Day,1984-03-27,Queensbridge,"Queensbridge and Astoria, too, the story is true",The young guns and Lost Boys made the news,So let's party for the borough and shit,0.00,0.223,0.777,-0.3182,1984.0,1980.0


In [30]:
positive.head(10)

,Unnamed: 0,Artist,Song Name,release date,GPE,Line,Line Before,Line After,pos,neg,neu,compound,year,decade
3253,3253,Busta Rhymes,Flipmode Bitch,2020-10-30,New York,We smashin’ New York to the next town next town,You niggas know we got the best sound best sound,We bang the street don’t need a test ground te...,0.472,0.135,0.393,0.7906,2020.0,2020.0
3252,3252,Busta Rhymes,Flipmode Bitch,2020-10-30,New York,We smashin’ New York to the next town next town,You niggas know we got the best sound best sound,We bang the street don’t need a test ground te...,0.472,0.135,0.393,0.7906,2020.0,2020.0
3251,3251,Busta Rhymes,Flipmode Bitch,2020-10-30,New York,We smashin’ New York to the next town next town,You niggas know we got the best sound best sound,We bang the street don’t need a test ground te...,0.472,0.135,0.393,0.7906,2020.0,2020.0
3875,3875,Action Bronson,Consensual Rape,2020-09-25,America,Trying to live American Dreams...Dusty Rhodes,The forty-four long surely got a trusty home,NaN,0.550,0.000,0.450,0.7269,2020.0,2020.0
4340,4340,Ol' Dirty Bastard,Reunited,2020-03-28,West Coast,"MC Eiht, West Coast, word is bond","Give Snoop my love, E-40","This is family, I don't give a fuck!",0.512,0.000,0.488,0.6369,2020.0,2020.0
3349,3349,Nas,96 Freestyle,2021-08-06,Queensbridge,We\'ll strike back Queensbridge style,"It\'s like that, you know it\'s like that",My mobb deep people is mad foul,0.455,0.000,0.545,0.6124,2021.0,2020.0
3350,3350,Nas,96 Freestyle,2021-08-06,Queens,We\'ll strike back Queensbridge style,"It\'s like that, you know it\'s like that",My mobb deep people is mad foul,0.455,0.000,0.545,0.6124,2021.0,2020.0
3644,3644,A$AP Ferg,DJ Self Freestyle,2020-09-25,New York City,New York City!,Harlem! You should be proud!,NaN,0.479,0.000,0.521,0.5696,2020.0,2020.0
3643,3643,A$AP Ferg,DJ Self Freestyle,2020-09-25,New York,New York City!,Harlem! You should be proud!,NaN,0.479,0.000,0.521,0.5696,2020.0,2020.0
3874,3874,Action Bronson,Consensual Rape,2020-09-25,Queens,About to be a huge star and I'm still in Queens,Your life is cheap like a hooker in the Philip...,You're on the shelf like a figurine,0.238,0.000,0.762,0.3612,2020.0,2020.0


In [31]:
positive.to_csv('positive.csv')
negative.to_csv('negative.csv')

In [32]:
positive.columns

Index(['Unnamed: 0', 'Artist', 'Song Name', 'release date', 'GPE', 'Line',
       'Line Before', 'Line After', 'pos', 'neg', 'neu', 'compound', 'year',
       'decade'],
      dtype='object')

In [33]:
positive = positive.reindex(columns=['decade', 'compound', 'GPE','Unnamed: 0', 'Artist', 'Song Name', 'release date', 'Line',
       'Line Before', 'Line After', 'pos', 'neg', 'neu', 'year',
       ])

negative = negative.reindex(columns=['decade', 'compound', 'GPE','Unnamed: 0', 'Artist', 'Song Name', 'release date', 'Line',
       'Line Before', 'Line After', 'pos', 'neg', 'neu', 'year',
       ])


In [34]:
positive.to_csv('positive.csv')
negative.to_csv('negative.csv')